In [1]:
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GRU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

import matplotlib.pyplot as plt
%matplotlib inline

# Creating our Time Series Dataframe

In [2]:
infra_c = pd.read_csv('../Data/Infrastructure/Cleaned_Infrastructure/infra_clean.csv')

In [3]:
spend_c = pd.read_csv('../Data/Infrastructure/Cleaned_Infrastructure/hspend_clean.csv')

In [4]:
mex_pharm = pd.read_csv('../Data/Infrastructure/Cleaned_Infrastructure/pharm_spend_clean.csv')

In [5]:
tech_c = pd.read_csv('../Data/Infrastructure/Cleaned_Infrastructure/tech_clean.csv')

In [6]:
workers_c = pd.read_csv('../Data/Infrastructure/Cleaned_Infrastructure/workers_clean.csv')

In [7]:
mex_dr = pd.read_csv('../Data/Infrastructure/Cleaned_Infrastructure/doc_count_clean.csv')

In [8]:
mex_nurses = pd.read_csv('../Data/Infrastructure/Cleaned_Infrastructure/nurse_count_clean.csv')

In [9]:
hosp_c = pd.read_csv('../Data/Infrastructure/Cleaned_Infrastructure/hospcount_clean.csv')

In [10]:
cholera_c = pd.read_csv('../Data/Diseases/cleaned_disease/cholera_clean.csv')

In [11]:
malaria_c = pd.read_csv('../Data/Diseases/cleaned_disease/malaria_clean.csv')

In [12]:
measles_c = pd.read_csv('../Data/Diseases/cleaned_disease/measles_clean.csv')

In [13]:
mumps_c = pd.read_csv('../Data/Diseases/cleaned_disease/mumps_clean.csv')

In [14]:
pert_c = pd.read_csv('../Data/Diseases/cleaned_disease/pert_clean.csv')

In [15]:
rubella_c = pd.read_csv('../Data/Diseases/cleaned_disease/rubella_clean.csv')

In [16]:
tb_c = pd.read_csv('../Data/Diseases/cleaned_disease/tb_clean.csv')

In [17]:
tet_c = pd.read_csv('../Data/Diseases/cleaned_disease/tet_clean.csv')

# Filtering Infrastructure Data for Mexico

In [18]:
mex_infra = infra_c.loc[infra_c['Country Name'] == 'Mexico']

In [19]:
mex_infra.head()

,Country Name,Indicator Name,1999,2000,2001,2002,2003,2004,2005,2006,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
7144,Mexico,ICT goods exports (% of total goods exports),NaN,20.909537,22.061841,20.739979,19.311210,19.684402,17.988816,18.769373,...,20.166782,16.995108,16.858755,16.266397,16.038272,16.189341,16.143444,16.108171,NaN,NaN
7145,Mexico,ICT goods imports (% total goods imports),NaN,17.900664,18.906531,17.298672,17.392107,18.409806,17.142958,17.540997,...,19.225457,17.284306,16.507423,17.107871,16.338254,16.452883,16.508494,15.112958,NaN,NaN
7146,Mexico,Individuals using the Internet (% of population),1.857436,5.081384,7.038023,11.900000,12.900000,14.100000,17.210000,19.520000,...,31.050000,37.176295,39.750000,43.460000,44.390000,57.431043,59.540446,63.852249,65.772634,NaN
7147,Mexico,Secure Internet servers (per 1 million people),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,13.418882,17.295404,26.826030,31.608935,40.962110,57.886926,119.659418,185.482420,225.745480,NaN
7148,Mexico,Secure Internet servers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1531.000000,2001.000000,3146.000000,3756.000000,4930.000000,7054.000000,14758.000000,23144.000000,28487.000000,NaN


In [20]:
len(mex_infra['Indicator Name'].unique())

47

In [21]:
mex_infra.isnull().sum()

Country Name       0
Indicator Name     0
1999              22
2000              19
2001              17
2002              12
2003              19
2004              12
2005              14
2006              14
2007               4
2008              13
2009              15
2010              11
2011              11
2012               4
2013              11
2014               6
2015               7
2016              19
2017              19
2018              22
2019              47
dtype: int64

In [22]:
#figuring out which rows/indicators we have to lose because of nans
mex_infra

,Country Name,Indicator Name,1999,2000,2001,2002,2003,2004,2005,2006,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
7144,Mexico,ICT goods exports (% of total goods exports),NaN,2.090954e+01,2.206184e+01,2.073998e+01,1.931121e+01,1.968440e+01,1.798882e+01,1.876937e+01,...,2.016678e+01,1.699511e+01,1.685876e+01,1.626640e+01,1.603827e+01,1.618934e+01,1.614344e+01,1.610817e+01,NaN,NaN
7145,Mexico,ICT goods imports (% total goods imports),NaN,1.790066e+01,1.890653e+01,1.729867e+01,1.739211e+01,1.840981e+01,1.714296e+01,1.754100e+01,...,1.922546e+01,1.728431e+01,1.650742e+01,1.710787e+01,1.633825e+01,1.645288e+01,1.650849e+01,1.511296e+01,NaN,NaN
7146,Mexico,Individuals using the Internet (% of population),1.857436e+00,5.081384e+00,7.038023e+00,1.190000e+01,1.290000e+01,1.410000e+01,1.721000e+01,1.952000e+01,...,3.105000e+01,3.717630e+01,3.975000e+01,4.346000e+01,4.439000e+01,5.743104e+01,5.954045e+01,6.385225e+01,6.577263e+01,NaN
7147,Mexico,Secure Internet servers (per 1 million people),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.341888e+01,1.729540e+01,2.682603e+01,3.160893e+01,4.096211e+01,5.788693e+01,1.196594e+02,1.854824e+02,2.257455e+02,NaN
7148,Mexico,Secure Internet servers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.531000e+03,2.001000e+03,3.146000e+03,3.756000e+03,4.930000e+03,7.054000e+03,1.475800e+04,2.314400e+04,2.848700e+04,NaN
7149,Mexico,Fixed broadband subscriptions (per 100 people),NaN,1.516686e-02,4.985137e-02,2.276506e-01,4.155673e-01,1.011609e+00,1.813451e+00,2.807731e+00,...,9.275651e+00,9.996994e+00,1.115103e+01,1.072808e+01,1.082839e+01,1.211053e+01,1.303147e+01,1.374537e+01,1.464298e+01,NaN
7150,Mexico,Fixed broadband subscriptions,NaN,1.500000e+04,5.000000e+04,2.314860e+05,4.283710e+05,1.057282e+06,1.922352e+06,3.020000e+06,...,1.058286e+07,1.156607e+07,1.307728e+07,1.274787e+07,1.303252e+07,1.475769e+07,1.607216e+07,1.715110e+07,1.847809e+07,NaN
7151,Mexico,Fixed telephone subscriptions (per 100 people),1.120932e+01,1.246885e+01,1.373320e+01,1.472697e+01,1.584197e+01,1.729249e+01,1.840667e+01,1.846529e+01,...,1.745826e+01,1.728400e+01,1.755527e+01,1.564754e+01,1.542077e+01,1.655277e+01,1.694255e+01,1.663167e+01,1.692792e+01,NaN
7152,Mexico,Fixed telephone subscriptions,1.092738e+07,1.233168e+07,1.377415e+07,1.497508e+07,1.633007e+07,1.807324e+07,1.951202e+07,1.986130e+07,...,1.991864e+07,1.999680e+07,2.058780e+07,1.859353e+07,1.855969e+07,2.017091e+07,2.089582e+07,2.075255e+07,2.136147e+07,NaN
7153,Mexico,Mobile cellular subscriptions (per 100 people),7.931116e+00,1.423448e+01,2.169288e+01,2.549868e+01,2.919810e+01,3.679009e+01,4.445890e+01,5.150184e+01,...,8.009564e+01,8.175190e+01,8.589039e+01,8.983415e+01,8.719889e+01,8.837176e+01,9.059238e+01,9.162671e+01,9.300776e+01,NaN


In [23]:
#dropping rows with too many null values
mex_infra = mex_infra.drop([7157, 7158, 7159, 7163, 7164, 7165, 7173, 7175, 7176, 7177, 7178, 7179, 7180, 7181])

In [42]:
mex_infra.drop(axis = 0, columns = ['Country Name'], inplace = True)

In [44]:
mex_infra = mex_infra.set_index('Indicator Name')

In [48]:
#checking to see if data is as expected
mex_infra.head()

,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
Indicator Name,,,,,,,,,,,,,,,,,,,,,
ICT goods exports (% of total goods exports),NaN,20.909537,22.061841,20.739979,19.311210,19.684402,17.988816,18.769373,17.713619,19.534464,...,20.166782,16.995108,16.858755,16.266397,16.038272,16.189341,16.143444,16.108171,NaN,NaN
ICT goods imports (% total goods imports),NaN,17.900664,18.906531,17.298672,17.392107,18.409806,17.142958,17.540997,13.505200,16.112432,...,19.225457,17.284306,16.507423,17.107871,16.338254,16.452883,16.508494,15.112958,NaN,NaN
Individuals using the Internet (% of population),1.857436,5.081384,7.038023,11.900000,12.900000,14.100000,17.210000,19.520000,20.810000,21.710000,...,31.050000,37.176295,39.750000,43.460000,44.390000,57.431043,59.540446,63.852249,65.772634,NaN
Secure Internet servers (per 1 million people),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,13.418882,17.295404,26.826030,31.608935,40.962110,57.886926,119.659418,185.482420,225.745480,NaN
Secure Internet servers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1531.000000,2001.000000,3146.000000,3756.000000,4930.000000,7054.000000,14758.000000,23144.000000,28487.000000,NaN


In [46]:
mex_infra.apply(lambda row: row.fillna(row.mean()), axis=1)

,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
Indicator Name,,,,,,,,,,,,,,,,,,,,,
ICT goods exports (% of total goods exports),1.850795e+01,2.090954e+01,2.206184e+01,2.073998e+01,1.931121e+01,1.968440e+01,1.798882e+01,1.876937e+01,1.771362e+01,1.953446e+01,...,2.016678e+01,1.699511e+01,1.685876e+01,1.626640e+01,1.603827e+01,1.618934e+01,1.614344e+01,1.610817e+01,1.850795e+01,1.850795e+01
ICT goods imports (% total goods imports),1.713035e+01,1.790066e+01,1.890653e+01,1.729867e+01,1.739211e+01,1.840981e+01,1.714296e+01,1.754100e+01,1.350520e+01,1.611243e+01,...,1.922546e+01,1.728431e+01,1.650742e+01,1.710787e+01,1.633825e+01,1.645288e+01,1.650849e+01,1.511296e+01,1.713035e+01,1.713035e+01
Individuals using the Internet (% of population),1.857436e+00,5.081384e+00,7.038023e+00,1.190000e+01,1.290000e+01,1.410000e+01,1.721000e+01,1.952000e+01,2.081000e+01,2.171000e+01,...,3.105000e+01,3.717630e+01,3.975000e+01,4.346000e+01,4.439000e+01,5.743104e+01,5.954045e+01,6.385225e+01,6.577263e+01,3.004448e+01
Secure Internet servers (per 1 million people),7.987618e+01,7.987618e+01,7.987618e+01,7.987618e+01,7.987618e+01,7.987618e+01,7.987618e+01,7.987618e+01,7.987618e+01,7.987618e+01,...,1.341888e+01,1.729540e+01,2.682603e+01,3.160893e+01,4.096211e+01,5.788693e+01,1.196594e+02,1.854824e+02,2.257455e+02,7.987618e+01
Secure Internet servers,9.867444e+03,9.867444e+03,9.867444e+03,9.867444e+03,9.867444e+03,9.867444e+03,9.867444e+03,9.867444e+03,9.867444e+03,9.867444e+03,...,1.531000e+03,2.001000e+03,3.146000e+03,3.756000e+03,4.930000e+03,7.054000e+03,1.475800e+04,2.314400e+04,2.848700e+04,9.867444e+03
Fixed broadband subscriptions (per 100 people),6.913037e+00,1.516686e-02,4.985137e-02,2.276506e-01,4.155673e-01,1.011609e+00,1.813451e+00,2.807731e+00,4.126043e+00,6.797468e+00,...,9.275651e+00,9.996994e+00,1.115103e+01,1.072808e+01,1.082839e+01,1.211053e+01,1.303147e+01,1.374537e+01,1.464298e+01,6.913037e+00
Fixed broadband subscriptions,8.203597e+06,1.500000e+04,5.000000e+04,2.314860e+05,4.283710e+05,1.057282e+06,1.922352e+06,3.020000e+06,4.504422e+06,7.532633e+06,...,1.058286e+07,1.156607e+07,1.307728e+07,1.274787e+07,1.303252e+07,1.475769e+07,1.607216e+07,1.715110e+07,1.847809e+07,8.203597e+06
Fixed telephone subscriptions (per 100 people),1.120932e+01,1.246885e+01,1.373320e+01,1.472697e+01,1.584197e+01,1.729249e+01,1.840667e+01,1.846529e+01,1.831805e+01,1.849152e+01,...,1.745826e+01,1.728400e+01,1.755527e+01,1.564754e+01,1.542077e+01,1.655277e+01,1.694255e+01,1.663167e+01,1.692792e+01,1.633595e+01
Fixed telephone subscriptions,1.092738e+07,1.233168e+07,1.377415e+07,1.497508e+07,1.633007e+07,1.807324e+07,1.951202e+07,1.986130e+07,1.999790e+07,2.049143e+07,...,1.991864e+07,1.999680e+07,2.058780e+07,1.859353e+07,1.855969e+07,2.017091e+07,2.089582e+07,2.075255e+07,2.136147e+07,1.833085e+07


**Notes**: Around 15/47 of our indicators cannot be used for this project. 

In [25]:
mex_spend = spend_c.loc[spend_c['LOCATION'] == 'MEX']

In [26]:
#percent gdp
mex_spend.head()

,LOCATION,INDICATOR,SUBJECT,MEASURE,TIME,Value
1279,MEX,HEALTHEXP,TOT,PC_GDP,1999,4.395
1280,MEX,HEALTHEXP,TOT,PC_GDP,2000,4.449
1281,MEX,HEALTHEXP,TOT,PC_GDP,2001,4.820
1282,MEX,HEALTHEXP,TOT,PC_GDP,2002,5.072
1283,MEX,HEALTHEXP,TOT,PC_GDP,2003,5.815


In [27]:
mex_pharm = mex_pharm.loc[mex_pharm['LOCATION'] == 'MEX']

In [28]:
mex_pharm.head()

,Unnamed: 0,LOCATION,INDICATOR,TIME,Value
333,333,MEX,PHARMAEXP,1999,88.193
334,334,MEX,PHARMAEXP,2000,100.973
335,335,MEX,PHARMAEXP,2001,110.626
336,336,MEX,PHARMAEXP,2002,127.529
337,337,MEX,PHARMAEXP,2003,242.818


In [29]:
mex_tech = tech_c.loc[tech_c['Country'] == 'Mexico']

In [30]:
mex_tech.head()

,Country,Variable,Year,Value
3330,Mexico,"Computed Tomography scanners, total",2001,250.0
3331,Mexico,"Computed Tomography scanners, total",2002,296.0
3332,Mexico,"Computed Tomography scanners, total",2003,309.0
3333,Mexico,"Computed Tomography scanners, total",2004,289.0
3334,Mexico,"Computed Tomography scanners, total",2005,325.0


In [31]:
mex_workers = workers_c.loc[workers_c['Country'] == 'Mexico']

In [32]:
mex_workers.drop(axis = 0, columns = ['YEA'], inplace = True)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [33]:
mex_workers.head()
#not enough data can't use. maybe fill in this data?

,Country,Variable,Year,Value
6468,Mexico,Medical graduates,2005,11973.0
6469,Mexico,Medical graduates,2006,10899.0
6470,Mexico,Medical graduates,2007,12255.0
6471,Mexico,Medical graduates,2008,13209.0
6472,Mexico,Medical graduates,2009,12926.0


In [34]:
mex_dr = mex_dr.loc[mex_dr['LOCATION'] == 'MEX']

In [35]:
#these are per 1000 people
mex_dr.head()

,Unnamed: 0,LOCATION,INDICATOR,TIME,Value
277,277,MEX,MEDICALDOC,1999,1.67
278,278,MEX,MEDICALDOC,2000,1.59
279,279,MEX,MEDICALDOC,2001,1.49
280,280,MEX,MEDICALDOC,2002,1.50
281,281,MEX,MEDICALDOC,2003,1.55


In [36]:
mex_nurses = mex_nurses.loc[mex_nurses['LOCATION'] == 'MEX']

In [37]:
#per 1000
mex_nurses.head()

,Unnamed: 0,LOCATION,INDICATOR,TIME,Value
263,263,MEX,NURSE,1998,2.09
264,264,MEX,NURSE,1999,2.14
265,265,MEX,NURSE,2000,2.19
266,266,MEX,NURSE,2001,2.19
267,267,MEX,NURSE,2002,2.19


In [38]:
mex_hosp =hosp_c.loc[hosp_c['Country'] == 'Mexico']

In [49]:
mex_hosp.head()

,Country,Variable,Year,Value
1097,Mexico,Hospitals,2000,3952.0
1098,Mexico,Hospitals,2001,3978.0
1099,Mexico,Hospitals,2002,4088.0
1100,Mexico,Hospitals,2003,4150.0
1101,Mexico,Hospitals,2004,4110.0


# Filtering Disease Data for Mexico

# Creating a Time Series Dataframe